1) Загрузите данные из файла data-logistic.csv. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('.../data-logistic.csv', header=None)
df.head(3)

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425


In [2]:
X = df.iloc[:, 1:]
#X1 = df.iloc[:, 1]
#X2 = df.iloc[:, 2]
y = df.iloc[:, 0]

2) Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

In [3]:
def func_for_w1(w1, w2, y, X, k, C):
    l = len(y)
    Summ = 0
    for i in range (0, l):
        Summ += y[i] * X[1][i] * (1.0 - 1.0 / (1.0 + np.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))
    return w1 + k * (1.0 / l) * Summ - k * C * w1

def func_for_w2(w1, w2, y, X, k, C):
    l = len(y)
    Summ = 0
    for i in range (0, l):
        Summ += y[i] * X[2][i] * (1.0 - 1.0 / (1.0 + np.exp(-y[i] * (w1 * X[1][i] + w2 * X[2][i]))))
    return w2 + k * (1.0 / l) * Summ - k * C * w2

3) Реализуйте градиентный спуск для обычной и L2-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага k=0.1. В качестве начального приближения используйте вектор (0, 0).

In [4]:
def grad_des(y, X, C=0.0, w1=0.0, w2=0.0, k=0.1, err=1e-5):
    i = 0
    i_max = 100000
    
    while True:
        i += 1
        w1_new, w2_new = func_for_w1(w1, w2, y, X, k, C), func_for_w2(w1, w2, y, X, k, C)
        evk = np.sqrt((w1_new - w1) ** 2 + (w2_new - w2) ** 2)
        
        if i >= i_max or evk <= err:
            break
        else:
            w1, w2 = w1_new, w2_new
    return [w1_new, w2_new]

4) Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [5]:
w1, w2 = grad_des(y, X)
w1_reg, w2_reg = grad_des(y, X, C=10.0)

5) Какое значение принимает AUC-ROC на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: a(x) = 1 / (1 + exp(-w1 x1 - w2 x2)).

In [6]:
def sigma(X, w1, w2):
    return 1.0 / (1.0 + np.exp(-w1 * X[1] - w2 * X[2]))
scores = X.apply(lambda x: sigma(x, w1, w2), axis=1)
scores_reg = X.apply(lambda x: sigma(x, w1_reg, w2_reg), axis=1)

In [7]:
from sklearn.metrics import roc_auc_score
RAS = roc_auc_score(y, scores)
RAS_reg = roc_auc_score(y, scores_reg)
print(RAS, RAS_reg)

0.926857142857 0.936285714286


6) Попробуйте поменять длину шага. Будет ли сходиться алгоритм, если делать более длинные шаги? Как меняется число итераций при уменьшении длины шага?

In [8]:
w1_test, w2_test = grad_des(y, X, k=1)
scores_test = X.apply(lambda x: sigma(x, w1_test, w2_test), axis=1)
RAS_test = roc_auc_score(y, scores_test)
RAS_test

0.92676190476190479

In [9]:
# Ничего не меняется

7) Попробуйте менять начальное приближение. Влияет ли оно на что-нибудь?

In [10]:
w1_test, w2_test = grad_des(y, X, w1=10, w2=10)
scores_test = X.apply(lambda x: sigma(x, w1, w2), axis=1)
RAS_test = roc_auc_score(y, scores_test)
RAS_test

0.92685714285714282

In [11]:
# Ничего не меняется